<a href="https://colab.research.google.com/github/tadie/Tade-file/blob/main/notebooks/3.0%20speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speech Recognition


## Imports

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('No GPU.')
else:
  print(gpu_info)

Mon Oct 17 07:27:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18167328350589874552
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14415560704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15263938289212951000
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
import os
import sys
import cv2
import pickle

import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.io as pio

import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import Image
import plotly.graph_objects as go
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle

from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard


import IPython.display as ipd
from wordcloud import WordCloud
import plotly.express as px
from mpl_toolkits.axes_grid1 import make_axes_locatable
import librosa.display
from pathlib import Path
import json
import pickle


In [5]:
import logging
import sys
from logging.handlers import TimedRotatingFileHandler
FORMATTER = logging.Formatter(
    "%(asctime)s — %(name)s — %(levelname)s — %(message)s")
LOG_FILE = "my_app.log"


def get_console_handler():
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(FORMATTER)
    return console_handler


def get_file_handler():
    file_handler = TimedRotatingFileHandler(LOG_FILE, when='midnight')
    file_handler.setFormatter(FORMATTER)
    return file_handler


def get_logger(logger_name):
    logger = logging.getLogger(logger_name)
    # better to have too much log than not enough
    logger.setLevel(logging.DEBUG)
    logger.addHandler(get_console_handler())
    logger.addHandler(get_file_handler())
    # with this pattern, it's rarely necessary to propagate the error up to parent
    logger.propagate = False
    return logger


def get_logger(logger_name):
    logger = logging.getLogger(logger_name)
    # better to have too much log than not enough
    logger.setLevel(logging.DEBUG)
    logger.addHandler(get_console_handler())
    logger.addHandler(get_file_handler())
    # with this pattern, it's rarely necessary to propagate the error up to parent
    logger.propagate = False
    return logger

In [8]:

get_logger
class AudioVis():
  """Visualisation of the audio file, spectogram, mfcc....
  """
  
  def __init__(self):
    self.logger = get_logger("FileHandler")

  def play_audio(self, samples, sr=22000):
    return ipd.Audio(samples, rate=sr)

  def wav_plot(self, signal, title, x_label, y_label, sr=22000):
    plt.figure(figsize=(25, 5))
    librosa.display.waveplot(signal, sr=sr)
    plt.title(title)
    plt.ylabel(x_label)
    plt.xlabel(y_label)
    plt.show()

  def get_wc(self, df, column, stop_words):
    plt.figure(figsize=(30, 20))
    wordcloud = WordCloud(font_path='../fonts/NotoSansEthiopic-Medium.ttf', max_words=5000,
                          background_color="salmon", width=3000, height=2000, colormap='Pastel1',
                          collocations=False, stopwords=stop_words).generate(' '.join(df[column].values))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title('Most Frequent Words In Amharic Audio Transcription', fontsize=16)
    plt.show()

  def plot_raw_audio(vis_raw_audio, title='Audio Signal', size=(12, 3)):
    fig = plt.figure(figsize=size)
    ax = fig.add_subplot(111)
    steps = len(vis_raw_audio)
    ax.plot(np.linspace(1, steps, steps), vis_raw_audio)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.show()

  def plot_mfcc_feature(self, vis_mfcc_feature):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(111)
    im = ax.imshow(vis_mfcc_feature, cmap=plt.cm.jet, aspect='auto')
    plt.title('Normalized MFCC')
    plt.ylabel('Time')
    plt.xlabel('MFCC Coefficient')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    ax.set_xticks(np.arange(0, 13, 2), minor=False)
    plt.show()

  def plot_spectrogram_feature(self, vis_spectrogram_feature):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(111)
    im = ax.imshow(vis_spectrogram_feature, cmap=plt.cm.jet, aspect='auto')
    plt.title('Normalized Spectrogram')
    plt.ylabel('Time')
    plt.xlabel('Frequency')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    plt.show()

In [9]:
get_logger
class CleanAudio():
  """Clean audio data by removing dead spaces, ...
  """

  def __init__(self):
    self.logger = get_logger("CleanAudio")

  def normalize_audio(self, signal):
    feats_mean = np.mean(signal, axis=0)
    feats_std = np.std(signal, axis=0)
    signal = (signal - feats_mean) / (feats_std + 1e-14)
    return signal

  def trim_audio(self, signal, trim_db=None):
    signal, index = librosa.effects.trim(signal, top_db=trim_db)
    return signal

  def split_audio(self, signal, clean_db=None):
    yt = librosa.effects.split(signal, top_db=clean_db)
    cleaned_signal = []
    for start_i, end_i in yt:
      cleaned_signal.append(signal[start_i: end_i])
    signal = np.concatenate(np.array(cleaned_signal), axis=0)
    return signal

In [10]:
class Config:
  RANDOM_SEED = 27
  ASSETS_PATH = Path("../")
  REPO = "https://github.com/10acad-group3/speechTotext"
  DATASET_PATH = ASSETS_PATH / "data"
  FEATURES_PATH = ASSETS_PATH / "features"
  MODELS_PATH = ASSETS_PATH / "models"
  METRICS_FILE_PATH = ASSETS_PATH / "metrics"
  IMAGE_PATH = ASSETS_PATH / "img"

In [88]:

import json
import pickle
Config
get_logger
from tqdm import tqdm
CleanAudio


PATH_TRAIN_WAV = "/content/drive/MyDrive/data/AMHARIC/train/wav/"
PATH_TEST_WAV = "/content/drive/MyDrive/data/AMHARIC/test/wav/"
CLEAN_PATH_TRAIN_WAV = "/content/drive/MyDrive/data/AMHARIC_CLEAN/train/wav/"
CLEAN_PATH_TEST_WAV = "/content/drive/MyDrive/data/AMHARIC_CLEAN/test/wav/"


class FileHandler():
  """Read audio, audio transcription, Save cleaned Audio and transcriptions
  """

  def __init__(self):
    self.logger = get_logger("FileHandler")
    self.clean_audio = CleanAudio()

  def save_csv(self, df, csv_path, index=False):
    try:
      df.to_csv(csv_path, index=index)
      self.logger.info("file saved as csv")

    except Exception:
      self.logger.exception("save failed")

  def read_csv(self, csv_path, missing_values=[]):
    try:
      df = pd.read_csv(csv_path, na_values=missing_values)
      self.logger.debug("file read as csv")
      return df
    except FileNotFoundError:
      self.logger.exception("file not found")

  def read_text(self, text_path):
    text = []
    with open(text_path) as fp:
      line = fp.readline()
      while line:
        text.append(line)
        line = fp.readline()
    return text

  def read_data(self, PATH_TRAIN_TEXT, PATH_TEST_TEXT, train_labels, test_labels):
    train_text = self.read_text(PATH_TRAIN_TEXT)
    test_text = self.read_text(PATH_TEST_TEXT)

    train_text.extend(test_text)
    train_labels.extend(test_labels)

    new_text = []
    new_labels = []
    for i in train_text:
      result = i.split()

      if result[0] in train_labels:  # if the audio file exists
        new_text.append(' '.join([elem for elem in result[1:]]))
        new_labels.append(result[0])

    return new_text, new_labels

  def read_audio_signal(self, audio_file_loc, sr=22000):
    samples, sample_rate = librosa.load(audio_file_loc, sr=sr)
    return (samples, sample_rate)

  def save_audio_as_numpy(self, df, sr):
    inFiles = []
    outFiles = []
    for index, row in df.iterrows():
      if(row["category"] == "Train"):
        inFiles.append(PATH_TRAIN_WAV + row["key"] + ".wav")
        outFiles.append(CLEAN_PATH_TRAIN_WAV + row["key"] + ".npy")
      else:
        inFiles.append(PATH_TEST_WAV + row["key"] + ".wav")
        outFiles.append(CLEAN_PATH_TEST_WAV + row["key"] + ".npy")

    for in_file, out_file in zip(tqdm(inFiles), tqdm(outFiles)):
      try:
        wav, rate = librosa.load(in_file, sr=None)
        y = librosa.resample(wav, rate, sr)

        y = self.clean_audio.normalize_audio(y)
        y = self.clean_audio.split_audio(y, 30)

        np.save(out_file, y)

      except EOFError as e:
        self.logger = get_logger("Failed to save audio \n" + e)

In [92]:
class LogMelgramLayer(Layer):
  def __init__(self, num_fft, hop_length, num_mels, sample_rate, f_min, f_max, eps, **kwargs):
    super(LogMelgramLayer, self).__init__(**kwargs)

    self.num_fft = num_fft
    self.hop_length = hop_length
    self.num_mels = num_mels
    self.sample_rate = sample_rate
    self.f_min = f_min
    self.f_max = f_max
    self.eps = eps
    self.num_freqs = num_fft // 2 + 1
    lin_to_mel_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=self.num_mels,
        num_spectrogram_bins=self.num_freqs,
        sample_rate=self.sample_rate,
        lower_edge_hertz=self.f_min,
        upper_edge_hertz=self.f_max,
    )

    self.lin_to_mel_matrix = lin_to_mel_matrix

  def build(self, input_shape):
    self.non_trainable_weights.append(self.lin_to_mel_matrix)
    super(LogMelgramLayer, self).build(input_shape)

  def call(self, input):

    def _tf_log10(x):
      numerator = tf.math.log(x)
      denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
      return numerator / denominator

    stfts = tf.signal.stft(
        input,
        frame_length=self.num_fft,
        frame_step=self.hop_length,
        pad_end=False,  # librosa test compatibility
    )
    mag_stfts = tf.abs(stfts)

    melgrams = tf.tensordot(  # assuming channel_first, so (b, c, f, t)
        tf.square(mag_stfts), self.lin_to_mel_matrix, axes=[2, 0]
    )
    log_melgrams = _tf_log10(melgrams + self.eps)
    return tf.expand_dims(log_melgrams, 3)

  def get_config(self):
    config = {
        'num_fft': self.num_fft,
        'hop_length': self.hop_length,
        'num_mels': self.num_mels,
        'sample_rate': self.sample_rate,
        'f_min': self.f_min,
        'f_max': self.f_max,
        'eps': self.eps,
    }
    base_config = super(LogMelgramLayer, self).get_config()
    return dict(list(config.items()) + list(base_config.items()))

In [90]:
sns.set()
plt.style.use('ggplot')
%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [91]:
clean_audio = CleanAudio()
file_handler = FileHandler()
audio_vis = AudioVis()

## Load Data


In [111]:
# # Vs Code
# PATH_Model = "../models/"
# PATH_IMG = "../img/"

PATH_Model = "/content/drive/MyDrive/PATH_Model/"
PATH_IMG = "/content/drive/MyDrive/imgs/"
PATH_TRAIN_WAV = "/content/drive/MyDrive/data/AMHARIC_CLEAN/train/wav/"
PATH_TEST_WAV = "/content/drive/MyDrive/data/AMHARIC_CLEAN/test/wav/"
data = pd.read_csv('/content/drive/MyDrive/data/clean_data.csv')

In [96]:
data.head(5)

,Unnamed: 0,key,text,char_length,duration,speed,category
0,0,tr_10000_tr097082,የተለያዩ የትግራይ አውራጃ ተወላጆች ገንዘባቸውን አዋጥተው የልማት ተቋማትን እንዲመሰርቱ ትልማ አይፈቅድ ም,67,7.42,9.02,Train
1,1,tr_10001_tr097083,የጠመንጃ ተኩስ ተከፈተና አራት የኤርትራ ወታደሮች ተገደሉ,36,4.67,7.71,Train
2,2,tr_10002_tr097084,ላነሷቸው ጥያቄዎች የሰጡትን መልስ አቅርበ ነዋል,30,4.67,6.42,Train
3,3,tr_10003_tr097085,እብዱ አስፋልቱ ላይየ ኰለኰ ለው ድንጋይ መኪና አላሳልፍ አለ,38,4.42,8.61,Train
4,4,tr_10004_tr097086,ጠጁን ኰ መኰ መ ኰ መኰ መና ሚስቱን ሲ ያሰቃያት አደረ,35,4.22,8.29,Train


In [97]:
def get_paths(df):
  paths = []
  for col, row in df.iterrows():
    if(row["category"] == "Train"):
      paths.append(PATH_TRAIN_WAV + row["key"] + ".npy")
    else:
      paths.append(PATH_TEST_WAV + row["key"] + ".npy")

  return paths

In [98]:
data["path"] = get_paths(data)
data.sort_values(by=["duration"], inplace=True)
data.reset_index(drop=True, inplace=True)
data = data[["text", "char_length", "duration", "path"]]
data[["text", "char_length", "duration"]]

,text,char_length,duration
0,ሼራተን አዲስ ተመረቀ,13,2.05
1,ፖሊሱ እስረኞቹን ቆጠረ,14,2.05
2,ይህ ቀስ በቀስ እያደገ ሄደ,17,2.05
3,ኢነጋማ ህጋዊ እውቅና አገኘ,17,2.05
4,በተጨባጭ ስና የው ግን ባዶ ነው,20,2.05
...,...,...,...
10669,ቦናፓርቲ ያዊ ያልሆኑ ብዛት ያላቸው ጸረ ህዝብ መንግስታት በጸረ ዴሞከራሲ ና በሙስና ውስጥ ሲዘፈቁ እንደሚታዩ ለማወቅ ትንሽ አስተውሎት ን ነው የሚ ጠይቀው,98,13.95
10670,ከዚህ እጅግዘመናዊ ና ውድ ሰአት ሽያጭ ትርፍ ሶስት በመቶ በቋሚነት ሳኦ ክሪስቶ ቮል ፋውንዴሽን ለተባለው የሮናልዶ የእርዳታ ድርጅት ይው ላል,89,13.95
10671,ይልቁንም በተለመደው አኳኋን ከሚኒስትሩ በታች መሆን ያለበት ኤታማዦር ሹም በቀጥታ ሪፖርት የሚያደርገውና ተጠሪነቱ ለጠቅላይ ሚኒስትሩ ነው,86,13.95
10672,ፕሮጀክቱን ለማዘጋጀትና ለማቀነባበር እንዲሁም ጥናቱና ዲዛይኑን ተግባራዊ ለማድረግ እንዲቻል ከሶስቱም ሀገሮች የተውጣጡ ባለሙያዎች ያሉት የፕሮጀክት ጽፈት ቤት እንደሚኖርም ሚኒስትሩ ገልጸዋል,119,13.95


## Tokenizer

In [112]:
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, len_sent, filters, reverse=False):
        Tokenizer.__init__(self, filters=filters, char_level=True)

        self.len_sent = len_sent
        self.fit_on_texts(texts)

        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'
        else:
            truncating = 'post'

        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=len_sent,
                                           padding=padding,
                                           truncating=truncating
                                           )

    def token_to_word(self, token):
        word = " " if token == 0 else self.index_to_word[token]
        return word

    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = "".join(words)
        return text

    def text_to_tokens(self, text, reverse=False, padding=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'

        if padding:
            tokens = pad_sequences(tokens,
                                   maxlen=self.len_sent,
                                   padding=truncating,
                                   truncating=truncating
                                   )
        return tokens


In [100]:
MAX_SENTENCE_LENGTH = 125       # The longest sentence in the data is around 150 chars
filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n።”፤፦’፥'  # { ።”፤፦’፥' } unique for amharic

In [101]:
%%time
tokenizer = TokenizerWrap(texts=data.text,
                          padding='post',
                          reverse=False,
                          len_sent=MAX_SENTENCE_LENGTH,
                          filters=filters)

CPU times: user 399 ms, sys: 3.69 ms, total: 403 ms
Wall time: 405 ms


In [102]:
print(len(tokenizer.word_index))
print(tokenizer.word_index)

222
{' ': 1, 'ን': 2, 'ት': 3, 'ው': 4, 'ስ': 5, 'ያ': 6, 'የ': 7, 'ተ': 8, 'በ': 9, 'አ': 10, 'ል': 11, 'እ': 12, 'ለ': 13, 'ር': 14, 'መ': 15, 'ም': 16, 'ች': 17, 'ና': 18, 'ደ': 19, 'ነ': 20, 'ገ': 21, 'ማ': 22, 'ባ': 23, 'ይ': 24, 'ሚ': 25, 'ግ': 26, 'ራ': 27, 'ቸ': 28, 'ላ': 29, 'ብ': 30, 'ድ': 31, 'ረ': 32, 'ሰ': 33, 'ከ': 34, 'ወ': 35, 'ኢ': 36, 'ታ': 37, 'ዳ': 38, 'ክ': 39, 'ዮ': 40, 'ዋ': 41, 'ህ': 42, 'ጵ': 43, 'ጥ': 44, 'ቀ': 45, 'ሪ': 46, 'ጠ': 47, 'ቅ': 48, 'ዲ': 49, 'ሳ': 50, 'ዎ': 51, 'ሮ': 52, 'ሩ': 53, 'ሉ': 54, 'ሆ': 55, 'ሁ': 56, 'ጋ': 57, 'ሊ': 58, 'ቶ': 59, 'ካ': 60, 'ፈ': 61, 'ጣ': 62, 'ፍ': 63, 'ሀ': 64, 'ሞ': 65, 'ሽ': 66, 'ዊ': 67, 'ዘ': 68, 'ቱ': 69, 'ሬ': 70, 'ኤ': 71, 'ኮ': 72, 'ሎ': 73, 'ኛ': 74, 'ዛ': 75, 'ሲ': 76, 'ቃ': 77, 'ጉ': 78, 'ቡ': 79, 'ቻ': 80, 'ዝ': 81, 'ፕ': 82, 'ቢ': 83, 'ዚ': 84, 'ኑ': 85, 'ሙ': 86, 'ሶ': 87, 'ጀ': 88, 'ቁ': 89, 'ኖ': 90, 'ኩ': 91, 'ቋ': 92, 'ሌ': 93, 'ቤ': 94, 'ሱ': 95, 'ኒ': 96, 'ቹ': 97, 'ኝ': 98, 'ጸ': 99, 'ዱ': 100, 'ቲ': 101, 'ጅ': 102, 'ሸ': 103, 'ዜ': 104, 'ቴ': 105, 'ቆ': 106, 'ዙ': 107, 'ዴ': 108, 'ኔ': 109, 'ጡ': 110, 'ኙ'

In [103]:
data.text[1]

'ፖሊሱ እስረኞቹን ቆጠረ'

In [104]:
sample = tokenizer.text_to_tokens(data.text[1], padding=True)
sample

array([[128,  58,  95,   1,  12,   5,  32, 121,  97,   2,   1, 106,  47,
         32,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [105]:
print(tokenizer.tokens_to_string(sample[0]))

ፖሊሱ እስረኞቹን ቆጠረ


save token

In [115]:
with open( str(PATH_Model + 'char_tokenizer_amharic.pickle'), 'wb') as handle:
  pickle.dump(tokenizer, handle)

PicklingError: ignored

## Data Augmentation

Next, we can do data augmentation on the raw audio signal by changing the Pitch or Audio Speed and adding noise by a random amount. Data augmentation adds more variety to our input data and helps the model to generalize on a wider range of inputs.

In [107]:
class AudioAugment():
  def __init__(self):
    pass
   
  def change_speed(self, data):
    speed_rate = np.random.uniform(0.8, 1.2)
    wav_speed_tune = cv2.resize(data, (1, int(len(data) * speed_rate))).squeeze()

    if len(wav_speed_tune) < len(data):
      padding = len(data) - len(wav_speed_tune)
      offset = padding // 2
      wav_speed_tune = np.pad(wav_speed_tune, (offset, padding - offset), "constant")
    else:
      wav_speed_tune = wav_speed_tune[:len(data)]

    return wav_speed_tune

  def add_noise(self, data, noise_levels=(0, 0.3)):
    noise_level = np.random.uniform(*noise_levels)
    noise = np.random.randn(len(data))
    data_noise = data + noise_level * noise

    return data_noise

  def change_pitch(self, data):
    n_steps = np.random.randint(-1, 2)
    return librosa.effects.pitch_shift(data, 8000, n_steps)


## DataGenerator


Now that we have completed all the pre - processing steps, we will define a Custom data generation Class using Keras.


Rather than resizing all the audio samples to have the same length by padding them with silence. The class will pad the audio and transcription based on the longest in the batch. Since padding can affect the way the networks function and can make a great deal when it comes to the performance and accuracies of our model. Sorting our data based on duration and generating each bach with small padding will allow us to minimize the downside of padding.

The class returns three augmented versions for a single audio file. Since we are augmenting the audio every time rather than saving the augmented file, our data will change randomly for every batch. For instance, the speed of the audio can speed up or slow down randomly. This will considerably increase the variability in our input data.


In [36]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, sr, batch_size=32, shuffle=True):
        self.data = data      # Data Augmentation
        self.sr = sr
        self.batch_size = batch_size / 4      # Data Augmentation
        self.audio_augment = AudioAugment()
        self.len = int(np.floor(data.shape[0]/ self.batch_size))
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return self.len

    def __data_generation(self, batch_data):

        longest_audio = int(batch_data["duration"].max() * self.sr)
        longest_trans = int(batch_data["char_length"].max())

        X_audio = np.zeros([int(self.batch_size * 4), longest_audio], dtype="float32")
        y_trans = np.ones([int(self.batch_size * 4), longest_trans], dtype="int64")
        X_length = np.ones([int(self.batch_size * 4), 1], dtype="int64") * longest_audio
        y_length = np.zeros([int(self.batch_size * 4), 1], dtype="int64")

        i = 0
        for col, row in batch_data.iterrows():

            # Add transcription
            transcription = tf.convert_to_tensor(tokenizer.text_to_tokens(row["text"], padding=True)[:, :longest_trans])
            y_trans[i,] = y_trans[i + 1,] = y_trans[i + 2,] = y_trans[i + 3,] = transcription
            y_length[i] = y_length[i + 1] = y_length[i + 2] = y_length[i + 3] = row["char_length"]

            # Add original Audio
            audio_length = int(row["duration"] * self.sr)
            wav = np.load(row["path"])[:audio_length]
            X_audio[i, :audio_length] = wav
            i += 1

            # Add noise
            wav_ = self.audio_augment.add_noise(wav)
            X_audio[i, :audio_length] = wav_
            i += 1

            # Add noise
            wav_ = self.audio_augment.add_noise(wav)
            X_audio[i, :audio_length] = wav_
            i += 1

            # # Pitch change
            # wav_ = self.audio_augment.change_pitch(wav)
            # X_audio[i, :audio_length] = wav_
            # i+=1

            # Speed change
            wav_ = self.audio_augment.change_speed(wav)
            X_audio[i, :audio_length] = wav_
            i += 1

        outputs = {'ctc': tf.zeros(([int(self.batch_size * 4)]), dtype=tf.dtypes.float32)}
        inputs = {
            'the_input': tf.convert_to_tensor(X_audio),
            'the_labels': tf.convert_to_tensor(y_trans),
            'input_length': tf.convert_to_tensor(X_length, dtype="float32"),
            'label_length': tf.convert_to_tensor(y_length)
        }
        return (inputs, outputs)

    def on_epoch_end(self):

        self.indexes = np.arange(self.len * self.batch_size)

        if self.shuffle == True:

            self.indexes = self.indexes.reshape(int(self.len), int(self.batch_size))
            np.random.shuffle(self.indexes)

            for i in range(self.len):
                np.random.shuffle(self.indexes[i])

            self.indexes = self.indexes.reshape(int(self.len * self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[int(index * self.batch_size):int((index + 1) * self.batch_size)]
        batch_data = self.data.iloc[indexes]
        return self.__data_generation(batch_data)


In [37]:
sr = 8000
batch_size = 128
sample_generator = DataGenerator(data, sr, batch_size, False)

In [38]:
sample_generator.__len__()

333

In [109]:
%%time
sample_data = sample_generator.__getitem__(261)

FileNotFoundError: ignored

In [108]:
sample_audios = sample_data[0]["the_input"]
sample_labels = sample_data[0]["the_labels"]
sample_audios_length = sample_data[0]["input_length"]
sample_labels_length = sample_data[0]["label_length"]

NameError: ignored

In [42]:
print(sample_audios.shape)
print(sample_labels.shape)
print(sample_audios_length.shape)
print(sample_labels_length.shape)

NameError: ignored

In [43]:
sample_labels[0]


NameError: ignored

In [44]:
print(tokenizer.tokens_to_string(sample_labels[0].numpy()))
audio_vis.play_audio(sample_audios[0], sr)

NameError: ignored

## Log Melgram


The spectrogram is a 2D plot between time and frequency where each point in the plot represents the amplitude of a particular frequency at a particular time in terms of intensity of color. In simple terms, the spectrogram is a spectrum(broad range of colors) of frequencies as it varies with time.

A Mel Spectrogram makes two important changes relative to a regular Spectrogram that plots Frequency vs Time. It uses the Mel Scale instead of Frequency on the y - axis and It uses the Decibel Scale instead of Amplitude to indicate colors.

There is an article on How to Easily Process Audio on Your GPU with TensorFlow by David Schwertfeger. The article describes how to leverage the power of the GPU to process audio data using the TensorFlow signal processor. I have used the code provided in the blog for generating Mel Spectrogram.


In [45]:
def preprocessin_model(fft_size, hop_size, n_mels, mfcc=False):

    input_data = Input(name='input', shape=(None,), dtype="float32")
    spec = LogMelgramLayer(
        num_fft=fft_size,
        hop_length=hop_size,
        num_mels=n_mels,
        sample_rate=sr,
        f_min=0.0,
        f_max=sr // 2,
        eps=1e-6)(input_data)
    x = BatchNormalization(axis=2)(spec)
    # x = Permute((2, 1, 3), name='permute', dtype="float32")(x)
    model = Model(inputs=input_data, outputs=x, name="preprocessin_model")

    return model

### hop_size and  n_mels choise


In [46]:
def compare(i, fft_size, n_mels_list, hop_size_list, sr=16000):

    sample_data = sample_generator.__getitem__(i)
    sample_audios = sample_data[0]["the_input"]
    sample_labels = sample_data[0]["the_labels"]

    nrows, ncols = len(hop_size_list), len(n_mels_list),
    plt.figure(figsize=(4 * nrows, 4 * ncols))

    for i in range(nrows):
        n_mels = n_mels_list[i]

        for y in range(ncols):
            hop_size = hop_size_list[y]

            plt.subplot(nrows, ncols, i * ncols + y + 1)

            model = preprocessin_model(fft_size, hop_size, n_mels)
            pred = model.predict(sample_audios)

            pred = pred[0, :, :, 0]
            librosa.display.specshow(pred.T, sr=sr, hop_length=hop_size, cmap="magma")
            plt.title('hop: {}, n_mels: {}, shape: {}'.format(hop_size, n_mels, pred.shape), fontsize=11)

    print("The longest sentence in this batch has {} characters".format(sample_labels.shape[1]))

    plt.tight_layout()
    plt.show()

In [47]:
fft_size = 256
n_mels_list = [256, 160, 128, 64]
hop_size_list = [256, 160, 128, 64]
compare(20, fft_size, n_mels_list, hop_size_list, sr)

FileNotFoundError: ignored

## Final Choice

In [48]:
fft_size = 256
hop_size = 128
n_mels = 128

In [49]:
melspecModel = preprocessin_model(fft_size, hop_size, n_mels)
melspecModel.summary()

Model: "preprocessin_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None)]            0         
                                                                 
 log_melgram_layer (LogMelgr  (None, None, 128, 1)     0         
 amLayer)                                                        
                                                                 
 batch_normalization (BatchN  (None, None, 128, 1)     512       
 ormalization)                                                   
                                                                 
Total params: 512
Trainable params: 256
Non-trainable params: 256
_________________________________________________________________


In [50]:
def vis(j=5):
    for i in range(0, 330, 330 // j):
        sample_data = sample_generator.__getitem__(i)
        sample_audios = sample_data[0]["the_input"]
        sample_labels = sample_data[0]["the_labels"]
        sample_labels_length = sample_data[0]["input_length"]

        melspec = melspecModel.predict(sample_audios)

        print('\n')
        print('-' * 100)

        print("The longest sentence in this batch has {} characters".format(sample_labels.shape[1]))
        print("We have to multiply the longest sentence by {} to reach length of Time steps".format(
            np.log2([melspec.shape[1] / sample_labels.shape[1]])[0]))

        print('-' * 100)
        print('\n')

        fig, ax = plt.subplots(figsize=(16, 4))
        pred = melspec[0, :, :, 0]
        vis_model(pred, "Mel-frequency spectrogram")

In [51]:
def vis_model(pred, title, cmap="magma"):
    librosa.display.specshow(pred.T, sr=sr, y_axis='mel', x_axis='time', hop_length=hop_size, cmap=cmap)
    plt.title('{}. Shape = {}'.format(title, pred.shape))
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()
    plt.show()

In [52]:
vis(5)

FileNotFoundError: ignored

In [60]:
melspec = melspecModel.predict(sample_audios)
melspec.shape

NameError: ignored

In [59]:
sample_audios.shape

NameError: ignored

## FINAL DATA

In [61]:
shuffled_data = shuffle(data)
shuffled_data.head()

TypeError: ignored

In [62]:
val_point = int(len(data)*.9)
test_point = int(len(data)*.95)
train_data = shuffled_data[:val_point]
val_data = shuffled_data[val_point:test_point]
test_data = shuffled_data[test_point:]

In [63]:
train_data = train_data.sort_values("duration")
val_data = val_data.sort_values("duration")
test_data = test_data.sort_values("duration")

In [64]:
sr = 8000
shuffle = True
batch_size = 32

In [65]:
train_gen = DataGenerator(train_data, sr, batch_size, shuffle)
val_gen = DataGenerator(val_data, sr, batch_size, shuffle)
test_gen = DataGenerator(test_data, sr, batch_size, shuffle)

In [66]:
print(f"Size of train data is {len(train_data)} and batch length is {len(train_gen)}")
print(f"Size of valid data is {len(val_data)}  and batch length is {len(val_gen)}")
print(f"Size of test data is {len(test_data)}  and batch length is {len(test_gen)}")

Size of train data is 9606 and batch length is 1200
Size of valid data is 534  and batch length is 66
Size of test data is 534  and batch length is 66


## CTC

We will use the CTC loss as the base loss function for our model. CTC is used to align the input and output sequences when the input is continuous and the output is discrete, and there are no clear element boundaries that can be used to map the input to the elements of the output sequence.

The Keras CTC loss function expects y_true, y_pred, input_length, and label_length as an argument. The problem is the input length changes for each batch in our data. We can compute the input length, by dividing the length of the audio by hop size.

The other point is if we are using Convolutional or Max Pooling layers in our model, we need to recompute input length since they can reduce the dimensions of the feature maps.

The final point to keep in mind is the input length(number of time slices) should be greater than two times transcription length plus One. This is mentioned in the CTC paper. Apart from these points, we can calculate CTC loss just by following the documentation from Keras.


In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
def input_lengths_lambda_func(args):
    input_length = args
    return tf.cast(tf.math.floor(input_length / hop_size)-1, dtype="float32")

In [ ]:
x = input_lengths_lambda_func(sample_audios_length[1]).numpy()
x

array([527.], dtype=float32)

In [ ]:
def add_ctc_loss(model_builder):
    the_labels = Input(name='the_labels', shape=(None,), dtype='float32')
    input_lengths = Input(name='input_length', shape=(1,), dtype='float32')
    label_lengths = Input(name='label_length', shape=(1,), dtype='float32')

    input_lengths2 = Lambda(input_lengths_lambda_func)(input_lengths)
    if model_builder.output_length:
        output_lengths = Lambda(model_builder.output_length)(input_lengths2)
    else:
        output_lengths = input_lengths2

    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
        [model_builder.output, the_labels, output_lengths, label_lengths])
    model = Model(inputs=[model_builder.input, the_labels, input_lengths, label_lengths], outputs=loss_out)
    return model

## Model Builder


In [ ]:
def build_model(output_dim, custom_model, calc=None):

    input_audios = Input(name='the_input', shape=(None,))
    pre = melspecModel(input_audios)
    pre.trainable = False  # Freeze the layer
    pre = tf.squeeze(pre, [3])

    y_pred = custom_model(pre)
    model = Model(inputs=input_audios, outputs=y_pred, name="model_builder")
    model.output_length = calc

    return model

## Model Trainer

In [ ]:
def train(model_builder,
          model_name,
          epochs,
          verbose=1,
          optimizer=SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
          ):

    model = add_ctc_loss(model_builder)

    # optimizer = Adam(lr=.01, clipnorm = 1, decay=1e-6)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    # print(model.summary())

    # add checkpointer
    checkpointer = ModelCheckpoint(filepath=PATH_Model+model_name+'.h5', verbose=0)
    early_stopping = EarlyStopping( monitor="val_loss", patience=5, restore_best_weights=True)

    # train the model
    hist = model.fit_generator(generator=train_gen,
                               validation_data=val_gen,
                               epochs=epochs,
                               callbacks=[checkpointer, early_stopping],
                               verbose=verbose,
                               use_multiprocessing=False)

    # save model loss
    with open(PATH_Model+model_name+'.pickle', 'wb') as f:
        pickle.dump(hist.history, f)


## Model Inference

In [69]:
from jiwer import wer

ModuleNotFoundError: ignored

In [67]:
def predict(data_gen,  num_elem=1, index=0):
    
    pred_data = data_gen.__getitem__(index)

    pred_audios = pred_data[0]["the_input"]
    pred_labels = pred_data[0]["the_labels"]
    pred_audios_length = pred_data[0]["input_length"]
    
    y_pred = model.predict(pred_audios)

    input_shape = tf.keras.backend.shape(y_pred)
    input_length = tf.ones(shape=input_shape[0]) * tf.keras.backend.cast(input_shape[1], 'float32')
    prediction = tf.keras.backend.ctc_decode(y_pred, input_length, greedy=False)[0][0]


    for i in range(0, num_elem):  # only on clean data
        
        pred = K.eval(prediction[i]).flatten().tolist()
        pred = list(filter(lambda a: a != -1, pred))

        ground_truth = tokenizer.tokens_to_string(pred_labels[i].numpy())
        hypothesis   = ''.join(tokenizer.tokens_to_string(pred))
        error        = wer(ground_truth, hypothesis)
                
        print('-'*48 + ' ' + str(i) + ' ' + '-'*48)
        print('True transcription:\n' + '\n' + ground_truth)
        print('-'*100)
        print('Predicted transcription:\n' + '\n' + hypothesis)
        print('-'*100)
        print('Word Error Rate:' + str(error))
        print('\n')

## Simple RNN

In [70]:
def simple_rnn_model(input_dim, output_dim=224):

    input_data = Input(name='the_input', shape=(None, input_dim))

    simp_rnn = GRU(output_dim, return_sequences=True, implementation=2, name='rnn')(input_data)

    y_pred = Activation('softmax', name='softmax')(simp_rnn)

    model = Model(inputs=input_data, outputs=y_pred, name="simple_rnn_model")

    model.output_length = lambda x: x

    return model

In [71]:
simple_rnn_model = simple_rnn_model(128, 224)
# plot_model(simple_rnn_model, to_file='../img/simple_rnn_model.png')
simple_rnn_model.summary()

Model: "simple_rnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 128)]       0         
                                                                 
 rnn (GRU)                   (None, None, 224)         237888    
                                                                 
 softmax (Activation)        (None, None, 224)         0         
                                                                 
Total params: 237,888
Trainable params: 237,888
Non-trainable params: 0
_________________________________________________________________


In [72]:
model = build_model(len(tokenizer.word_index)+2, simple_rnn_model)
model.summary()

NameError: ignored

In [73]:
# import mlflow
# mlflow.set_experiment('speech_recognition')
# mlflow.tensorflow.autolog()
train(model_builder=model, model_name="SimpleRNN_model", epochs=10)

NameError: ignored

In [74]:
x = predict(test_gen, 1, 0)

FileNotFoundError: ignored

## CONV_BRNN

$floor(\frac{n-f+2p}{s}+1) $

In [75]:
def cnn_output_length(input_length, kernel_list, pool_sizes, cnn_stride, mx_stride, padding='valid'):

    if padding == 'same':
        output_length = input_length
        
        return output_length

    elif padding == 'valid':

        output_length = input_length
        for i, j, k in zip(kernel_list, pool_sizes, mx_stride):
            output_length = (output_length - i)/cnn_stride + 1
            if j != 0: output_length = (output_length - j)/k + 1
        
        return tf.math.floor(output_length)

In [76]:
def CONV_BRNN( input_dim, filters, kernels, pool_sizes, mx_stride, cnn_stride, output_dim=224, num_cnn = 3, num_birnn = 4 ):

    input_data = Input(name='the_input', shape=(None, input_dim))
    x = Reshape((-1, input_dim, 1), dtype="float32")(input_data)
    
    for i in range(num_cnn):
        x = Conv2D(filters=filters[i], kernel_size=kernels[i], strides=1, padding='valid', name='cnn_{}'.format(i))(x)
        x = LeakyReLU(.1)(x)
        x = MaxPooling2D( pool_size=pool_sizes[i], strides=(1,2), padding="valid")(x)
        x = BatchNormalization(name='bn_cnn_{}'.format(i))(x)
        
    x = Reshape((-1, x.shape[-1] * x.shape[-2] ))(x)

    for i in range(num_birnn):
        x = Bidirectional(GRU(units=512, return_sequences=True, implementation=2, name='rnn_{}'.format(i)))(x)
        x = LeakyReLU(.1)(x)
        x = BatchNormalization(name='bn_rnn_{}'.format(i))(x)
  
    x = TimeDistributed(Dense(output_dim))(x)

    y_pred = Activation('softmax', name='softmax')(x)

    model = Model( inputs=input_data, outputs=y_pred, name="CONV_BRNN" )

    output_length_calculater = lambda x: cnn_output_length(x, kernels, pool_sizes, cnn_stride, mx_stride)
    
    return model, output_length_calculater

In [77]:
filters = [16, 32, 64]
kernels = [7, 5, 3] 
pool_sizes = [3, 3, 3]  
mx_stride = [1, 1, 2]
cnn_stride = 1

In [78]:
CONV_BRNN_Model, calc = CONV_BRNN(n_mels, filters, kernels, pool_sizes, mx_stride, cnn_stride,)
plot_model(CONV_BRNN_Model, to_file=PATH_Model+'CONV_BRNN_Model.png')
CONV_BRNN_Model.summary()

FileNotFoundError: ignored

In [ ]:
model = build_model(len(tokenizer.word_index)+2, CONV_BRNN_Model, calc)
model.summary()

In [ ]:
train(model_builder=model, model_name="CONV_BRNN_Model", epochs=10)

In [ ]:
x = predict(test_gen, 5, 0)

## RESNET

In [79]:
def cnn_output_length(input_length, kernel_list, pool_sizes, cnn_stride, mx_stride, padding='same'):

    if padding == 'same':        
        output_length = input_length
        for i, j in zip(cnn_stride, pool_sizes):
            output_length = (output_length)/i
            if j != 0: output_length = (output_length - j)/mx_stride + 1
                
        return tf.math.ceil(output_length)

    elif padding == 'valid':

        output_length = input_length
        for i, j in zip(kernel_list, pool_sizes):
            output_length = (output_length - i)/cnn_stride + 1
            if j != 0: output_length = (output_length - j)/mx_stride + 1
        
        return tf.math.floor(output_length)

In [80]:
def block(filters, inp):
    x = BatchNormalization()(inp)
    x = LeakyReLU(.1)(x)
    x = Dropout(.4)(x)
    x = Conv2D(filters, (3,3), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(.1)(x)
    x = Dropout(.4)(x)
    x = Conv2D(filters, (3,3), padding = 'same')(x)
    return(x)

In [81]:
def resnet( input_dim, output_dim=224, units=256,  num_birnn=2):

    filters = [32, 32, 32]   
    kernels = [3, 3, 3] 
    pool_sizes = [0, 0, 2]  
    cnn_stride = [1, 1, 1]
    mx_stride = 2

    input_data = Input(name='the_input', shape=(None, input_dim))
    x = Reshape((-1, input_dim, 1), dtype="float32")(input_data)
    
    x = Conv2D(filters[0], (3,3), padding = 'same')(x)
    x = MaxPooling2D((1,2), strides=(1, 2), padding = 'same')(x)

    x = Add()([block(filters[0], x),x])  
    x = Add()([block(filters[0], x),x])
    x = Add()([block(filters[0], x),x])

    x = Conv2D(filters[1], (3,3), padding = 'same')(x)
    x = MaxPooling2D((1,2), strides=(1, 2), padding = 'same')(x)
    
    x = Add()([block(filters[1], x),x])
    x = Add()([block(filters[1], x),x])
    x = Add()([block(filters[1], x),x])
    
    x = Conv2D(filters[2], (3,3), padding = 'same')(x)
    x = MaxPooling2D((1,2), strides=(1, 2), padding = 'same')(x)

    x = Add()([block(filters[2], x),x])  
    x = Add()([block(filters[2], x),x])
    x = Add()([block(filters[2], x),x])

    # x = MaxPooling2D((2,2), strides=2, padding = 'same')(x)
    x = AveragePooling2D((2, 2), strides=2, padding='same')(x)
    x = Reshape((-1, x.shape[-1] * x.shape[-2] ))(x)

    # GRULayer
    for i in range(num_birnn):
        x = Bidirectional(GRU(units=units, return_sequences=True, implementation=2, name='rnn_{}'.format(i)))(x)
        x = Dropout(.4)(x)
        x = LeakyReLU(.1)(x)
        x = BatchNormalization(name='bn_rnn_{}'.format(i))(x)

    x = TimeDistributed(Dense(output_dim))(x)
    y_pred = Activation('softmax', name='softmax')(x)

    model = Model( inputs=input_data, outputs=y_pred, name="custom_model" )

    output_length_calculater = lambda x: cnn_output_length(x, kernels, pool_sizes, cnn_stride, mx_stride)
    
    return model, output_length_calculater

In [82]:
resnet, calc = resnet(n_mels, 224, 512, 4)
# plot_model(resnet, to_file='/content/drive/MyDrive/resnet.png')
resnet.summary()

Model: "custom_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 the_input (InputLayer)         [(None, None, 128)]  0           []                               
                                                                                                  
 reshape_2 (Reshape)            (None, None, 128, 1  0           ['the_input[0][0]']              
                                )                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, None, 128, 3  320         ['reshape_2[0][0]']              
                                2)                                                                
                                                                                       

In [83]:
model = build_model(len(tokenizer.word_index)+2, resnet, calc)
model.summary()

NameError: ignored

In [84]:
# mlflow.set_experiment('Tensorflow')
# mlflow.tensorflow.autolog()
train(model_builder=model, model_name="resnet_Model", epochs=25)

NameError: ignored

In [85]:
file = open("/content/resnet_Model.pickle",'rb')
object_file = pickle.load(file)
file.close()

FileNotFoundError: ignored

In [86]:
  def plot_history(history):
    plt.figure(figsize=(12, 8))
    plt.plot(history['loss'], label="loss")
    plt.plot(history['val_loss'], label="val_loss")
    plt.legend()
    # time = strftime("%Y-%m-%d-%H:%M", gmtime())
    plt.show()

In [87]:
plot_history(object_file)

NameError: ignored

In [ ]:
train(model_builder=model, model_name="resnet_new", epochs=50)

Epoch 1/50
1200/1200 [==============================] - 738s 609ms/step - loss: 14.9248 - val_loss: 24.9730
Epoch 2/50
1200/1200 [==============================] - 732s 609ms/step - loss: 14.8936 - val_loss: 25.7370
Epoch 3/50
1200/1200 [==============================] - 735s 612ms/step - loss: 14.6069 - val_loss: 25.9315
Epoch 4/50
1200/1200 [==============================] - 722s 601ms/step - loss: 13.9830 - val_loss: 26.1137
Epoch 5/50
1200/1200 [==============================] - 720s 600ms/step - loss: 13.3778 - val_loss: 25.9429
Epoch 6/50
1200/1200 [==============================] - 727s 606ms/step - loss: 12.9830 - val_loss: 27.0448
Epoch 7/50
1200/1200 [==============================] - 726s 605ms/step - loss: 12.1761 - val_loss: 25.5905
Epoch 8/50
1200/1200 [==============================] - 735s 612ms/step - loss: 12.1303 - val_loss: 26.4370
Epoch 9/50
1200/1200 [==============================] - 732s 610ms/step - loss: 11.7117 - val_loss: 26.4420
Epoch 10/50
1200/1200 [=====

In [ ]:
opt = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
train(model_builder=model, model_name="resnet_v3", epochs=50, optimizer=opt)

Epoch 1/50
1200/1200 [==============================] - 742s 609ms/step - loss: 12.8161 - val_loss: 24.0373
Epoch 2/50
1200/1200 [==============================] - 738s 614ms/step - loss: 10.8825 - val_loss: 23.9008
Epoch 3/50
1200/1200 [==============================] - 736s 613ms/step - loss: 10.9055 - val_loss: 24.2030
Epoch 4/50
1200/1200 [==============================] - 734s 611ms/step - loss: 10.5747 - val_loss: 24.4141
Epoch 5/50
1200/1200 [==============================] - 740s 616ms/step - loss: 10.4964 - val_loss: 23.9909
Epoch 6/50
1200/1200 [==============================] - 723s 602ms/step - loss: 10.0709 - val_loss: 24.5373
Epoch 7/50
1200/1200 [==============================] - 734s 611ms/step - loss: 9.8915 - val_loss: 24.2895
Epoch 8/50
1200/1200 [==============================] - 735s 612ms/step - loss: 10.0206 - val_loss: 24.2571
Epoch 9/50
1200/1200 [==============================] - 744s 619ms/step - loss: 9.8068 - val_loss: 24.7826
Epoch 10/50
1200/1200 [=======

In [ ]:
model_name = "resnet_v3"
model.load_weights(PATH_Model + model_name + '.h5')

In [ ]:
x = predict(test_gen, 20, 3)

------------------------------------------------ 0 ------------------------------------------------
True transcription:

የመንግስት ጋዜጠኛው ኢህአዴግን በዘረኝነት ከሰሰ
----------------------------------------------------------------------------------------------------
Predicted transcription:

የመንግስት ጋዜጠኛው ኢህአደግን በዘረኝነት ከሶስ
----------------------------------------------------------------------------------------------------
Word Error Rate:0.4


------------------------------------------------ 4 ------------------------------------------------
True transcription:

ቤዛ ጐረ መሰ መሰለኝ ትእዛዝ አልቀበልም አለ
----------------------------------------------------------------------------------------------------
Predicted transcription:

ቤዛ ጐረ መሰ መሰለኝ ትእዛዝ አልቀበልም አለ
----------------------------------------------------------------------------------------------------
Word Error Rate:0.0


------------------------------------------------ 8 ------------------------------------------------
True transcription:

በእለቱ ኢንተር አንድ ለዜሮ

In [ ]:
x = predict(test_gen, 20, 4)

------------------------------------------------ 0 ------------------------------------------------
True transcription:

ለምን ኢትዮጵያ ውጊያውን ማጠናከር እንዳለባት ከላይ የተሰጡት ምክንያቶች አጠቃላይ ሁኔታውን የገመገሙ ናቸው
----------------------------------------------------------------------------------------------------
Predicted transcription:

ለምን ኢትዮጵያ ውጊያውን ማጠናከር እንዳለባት ከላይ የተሰጡት ምክንያቶች አጠቃላይ ሁኔታውን የገመገሙ ናቸው
----------------------------------------------------------------------------------------------------
Word Error Rate:0.0


------------------------------------------------ 4 ------------------------------------------------
True transcription:

እንደ ቢሮው መግለጫ ግንባታቸው የተጠናቀቀ ፕሮጄክቶች በግንባታ ላይ ካሉት አንድመቶ አስራስምንት ፕሮጄክቶች መካከል የሚጠቀሱ ናቸው
----------------------------------------------------------------------------------------------------
Predicted transcription:

እንደቢሮ መግለጫ ግንባታቸው የተጠናቀ ፕሮጀክቶች በግንባታ ላይ ካሉት አንድመቶ አስራስምንት ፕሮጀክቶች መካከል የሚጠቀሱ ናቸው
----------------------------------------------------------------------------------